# Segment

Segment mouse pixels from background pixels

In [31]:
PARAMS = {
    'session_name':'two_mice_v2',
    'background_session_name':'background',
    'copy_background_from_other_directory': '',
    'recompute_background':False,
    'working_directory':'long_pairs',
    'multiprocessing':False,
    'n_background_frames':100,
    'background_update_rate': 0.2,
    'batch_size':100,
    'min_depth_difference': 10,
    'max_depth_difference': 500,
    'min_color_component_size': 500,
    'max_color_component_brightness': 100,
    'max_color_component_aspect_ratio': 10,
    'max_color_component_island_size': 500,
    'skip_first_frames':100
}

In [4]:
import shutil, json, os, sys, time, pickle, h5py, cv2, numpy as np, matplotlib.pyplot as plt, h5py
from MoseqMulti_segment import *
from MoseqMulti_utils import *
%matplotlib inline

### Get initial background images

In [5]:
if len(PARAMS['copy_background_from_other_directory']) > 0:
    print('Copying background from '+ PARAMS['copy_background_from_other_directory'])
    os.system('cp '+PARAMS['copy_background_from_other_directory']+'/background.p '+PARAMS['working_directory']+'/') 

elif (not PARAMS['recompute_background']) and os.path.exists(PARAMS['working_directory']+'/background.p'):
    print('Loading background from '+PARAMS['working_directory'])
    color_background, depth_background = pickle.load(open(PARAMS['working_directory']+'/background.p','rb'))  
else:
    print('Calculating background')
    color_background, depth_background = get_background(PARAMS)
    pickle.dump((color_background, depth_background), open(PARAMS['working_directory']+'/background.p','wb'))

Loading background from long_pairs


### For each camera, do adaptive color and depth. Use color for adaptive masking

In [ ]:
segment(PARAMS)

Segmenting camera 816312062021
...on frame 0 out of 200
long_pairs/data/two_mice_v2_816312062021_color_mask.avi
...on frame 100 out of 200
long_pairs/data/two_mice_v2_816312062021_color_mask.avi
Segmenting camera 816312062005
...on frame 0 out of 200
long_pairs/data/two_mice_v2_816312062005_color_mask.avi
...on frame 100 out of 200
long_pairs/data/two_mice_v2_816312062005_color_mask.avi
Segmenting camera 739112061202
...on frame 0 out of 200
